# Cloning the Repository (pc-gym)

In [1]:
!dir pc-gym

 Volume in drive C is Windows 
 Volume Serial Number is F0B8-32A3

 Directory of c:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\train\base



File Not Found


In [2]:
# %cd pc-gym

# Importing Libraries

In [3]:
# %cd src

In [4]:
# %cd pcgym

In [5]:
%cd C:/Users/Usuario/Desktop/imperial_projects/VSCode/pcgym2/pc-gym

C:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\pc-gym


c:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\new_venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [20]:
from dataclasses import dataclass
import numpy as np
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
import torch.nn.functional as F
import copy
from stable_baselines3 import PPO,SAC,DDPG,TD3
import pcgym
from pcgym import make_env
import jax.numpy as jnp
#Global params
T = 26
nsteps =60
# Global seed for reproducibility
seed = 1990

# Saving and loading

Saving and loading stable-baselines models is straightforward: you can directly call `.save()` and `.load()` on the models.

In [21]:
import os
# Create save dir
save_dir = "./max/ddpg"
os.makedirs(save_dir, exist_ok=True)

# Monitoring experiments with W&B

In [8]:
!wandb login 84af17cc9914cf1736f3a8e2733a2f361e4750bb

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Usuario\.netrc


# 1.1 Reactor Case Study

In [5]:
# @title Function to log the performance data
def log_performance(performance, test_label, file_path):
    with open(file_path, "a") as file:
        file.write(f"{test_label}: \n")
        file.write(f"scalarised_performance: {performance}\n\n")

file_path = f"{save_dir}/lcb_metric_safe.txt"

### RL training

In [28]:
import numpy as np
import torch as th
from torch import nn  # Import torch.nn for activation functions

# Learning rate decay schedule
def cosine_annealing_schedule(progress_remaining: float, num_cycles=1, min_lr=0.005, max_lr=0.01):
    progress = 1.0 - progress_remaining
    lr = min_lr + (max_lr - min_lr) / 2 * (1 + np.cos(np.pi * num_cycles * progress))
    return lr

# Configuration for reinforcement learning model
config = {
    "policy": 'MlpPolicy',  # default: MlpPolicy
    "seed": 1990,
    "check_freq": 100,  # base: 12000 (~100 episodes)
    "n_eval_episodes": 10,  # evaluate the agent over 100 episodes in the evaluation environment
    "positive_definiteness_penalty_weight": 0,  # Set to 0 initially
    "derivative_penalty_weight": 0,  # Set to 0 initially
    'use_direct_penalty': False,  # choose between applying a penalty directly to the critic loss or adjusting the Q target values (derivative penalty)
    'allowed_increase_factor': 1,  # max increase value for both methods (derivative penalty)
    # 'total_timesteps': 500000,
    # 'action_noise_sigma': 0.34670933515233754,
}

# The best hyperparameters found in previous runs
best_params = {
    'min_lr': 0.0005334897747162678,
    'max_lr': 0.0003132010556972375,
    'pi_layer_0_units': 3,
    'pi_layer_1_units': 4,
    'qf_layer_0_units': 3,
    'qf_layer_1_units': 6,
    'activation_fn': 'LeakyReLU',
    'buffer_size': 200000,
    'batch_size': 512,
    'gamma': 0.9194039413768046,
    'tau': 0.012320756403750887,
    'learning_starts': 2082,
    'train_freq': 1,
    'action_noise_sigma': 0.34670933515233754,
    'total_timesteps': 50000,
}

# Update the config dictionary with the best parameters
config.update(best_params)

# Set the activation function directly based on best_params
if best_params['activation_fn'] == 'Tanh':
    activation_fn = th.nn.Tanh
elif best_params['activation_fn'] == 'ReLU':
    activation_fn = th.nn.ReLU
elif best_params['activation_fn'] == 'LeakyReLU':
    activation_fn = th.nn.LeakyReLU
else:
    raise ValueError("Unsupported activation function")

# # Create policy_kwargs with the fixed network architecture and activation function from best_params
policy_kwargs = dict(
    activation_fn=activation_fn,  # Use the activation function directly
    net_arch=dict(
        pi=[2 ** best_params['pi_layer_0_units'], 2 ** best_params['pi_layer_1_units']],
        qf=[2 ** best_params['qf_layer_0_units'], 2 ** best_params['qf_layer_1_units']]
    )
)

In [29]:
import torch as th
import numpy as np
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.vec_env import DummyVecEnv

##################################################################################
# Environment and RL Definition
##################################################################################

# Enter required setpoints for each state. Enter None for states without setpoints.
SP = {
    'T': [340.0 for _ in range(nsteps)],
}

# Continuous box action space
action_space = {
    'low': np.array([250]),
    'high': np.array([350])
}

# Continuous box observation space ([CA, T, CA_Setpoint, T_Setpoint])
observation_space = {
    'low': np.array([0.0, 300, 300]),
    'high': np.array([1, 450, 400])
}

r_scale = {
    'T': 1e-6  # Reward scale for each state,
}

# Define disturbance bounds
disturbance_bounds = {
    'low': np.array([330]),
    'high': np.array([370])
}

# Environment parameters
env_params_template = {
    'Nx': 2,
    'N': 60,
    'tsim': 26,
    'Nu': 1,
    'SP': SP,
    'o_space': observation_space,
    'a_space': action_space,
    'x0': np.array([0.87725294608097, 324.475443431599, 324.475443431599]),
    'model': 'cstr',
    'r_scale': r_scale,
    'normalise_a': True,
    'normalise_o': True,
    'noise': True,
    'integration_method': 'casadi',
    'noise_percentage': 0.001,  # 0.001,
    'disturbance_bounds': disturbance_bounds
}

# Add noise_percentage from env_params to config
config['noise_percentage'] = env_params_template['noise_percentage']

# Seed everything for reproducibility
def set_global_seeds(seed):
    np.random.seed(seed)
    th.manual_seed(seed)
    if th.cuda.is_available():
        th.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# Function to create random disturbances
def create_random_disturbances(seed, nsteps, low=330, high=370):
    # Set the global seed for reproducibility
    set_global_seeds(seed)
    value = np.random.uniform(low, high, 1)[0]  # Generate a single random disturbance value within the specified range
    disturbances = {'Ti': [350] * (nsteps // 3) + [value] * (nsteps // 3) + [350] * (nsteps // 3)}  # Repeat each disturbance value for nsteps/3 times
    return disturbances

# Create multiple environments with different disturbances
def create_parallel_envs(n_envs, seed):
    set_global_seeds(seed)
    envs = []
    disturbances_list = []
    for i in range(n_envs):
        env_params = env_params_template.copy()
        disturbances = create_random_disturbances(seed + i, nsteps)
        env_params.update({'disturbances': disturbances})
        disturbances_list.append(disturbances)
        envs.append(lambda: make_env(env_params))
    return DummyVecEnv(envs), disturbances_list

# Create evaluation environment using DummyVecEnv
def create_eval_env(seed, n_envs=1):
    set_global_seeds(seed)
    envs = []
    for i in range(n_envs):
        env_params = env_params_template.copy()
        disturbances = create_random_disturbances(seed + i, nsteps)
        env_params.update({'disturbances': disturbances})
        envs.append(lambda: make_env(env_params))
    return DummyVecEnv(envs)

eval_env = create_eval_env(seed=config['seed'], n_envs=1)

In [30]:

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.noise import OrnsteinUhlenbeckActionNoise

# Set the global seed for reproducibility
set_global_seeds(config['seed'])

# Number of parallel environments
n_envs = 10

# Create the parallel environments and get the disturbances
env, disturbances_list = create_parallel_envs(n_envs, config['seed'])

# Set up Ornstein-Uhlenbeck action noise after creating the environment
n_actions = env.action_space.shape
action_noise = OrnsteinUhlenbeckActionNoise(
    mean=np.zeros(n_actions), 
    sigma=config['action_noise_sigma'] * np.ones(n_actions)
)

# Print the first disturbances
print("First disturbance used in training:", disturbances_list[0])

model = DDPG(
    config['policy'],
    env,
    learning_rate=0.0001
    # learning_rate=lambda progress: cosine_annealing_schedule(progress, 
    #     min_lr=config['min_lr'], max_lr=config['max_lr']),  # Cosine schedule
    # batch_size=config['batch_size'],
    # gamma=config['gamma'],
    # tau=config['tau'],
    # buffer_size=config['buffer_size'],
    # learning_starts=config['learning_starts'],
    # train_freq=config['train_freq'],
    # action_noise=action_noise,
    # policy_kwargs=policy_kwargs,  # Use custom policy network structure
    seed=seed,
    verbose=1,
    # Adding the optimized DDPG-specific parameters to the model


)

# Create the evaluation environment
eval_env = create_eval_env(seed)


eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=save_dir,
    log_path=save_dir,
    eval_freq=config['check_freq'],
    n_eval_episodes=config['n_eval_episodes'],
    deterministic=True,
    render=False
)
# Train the model with the callbacks
model.learn(total_timesteps=config['total_timesteps'], callback=eval_callback)

# Save the model
model.save(save_dir)

# Save the disturbances to a file for future reference
disturbances_file_path = os.path.join(save_dir, "disturbances_used_in_training.txt")
with open(disturbances_file_path, "w") as f:
    for i, disturbance in enumerate(disturbances_list):
        f.write(f"Disturbance {i+1}: {disturbance}\n")

# Finish the W&B run


First disturbance used in training: {'Ti': [350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 358.8790902996886, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350]}
Using cuda device
-----------------------------
| time/              |      |
|    episodes        | 4    |
|    fps             | 1524 |
|    time_elapsed    | 0    |
|    total_timesteps | 590  |
-----------------------------
-----------------------------
| time/              |      |
|    episodes        | 8    |
|    fps             | 1520 |
|    time_elapsed    | 0   

C:\Users\mfb22\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=1000, episode_reward=-0.13 +/- 0.00
Episode length: 59.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 59       |
|    mean_reward     | -0.132   |
| time/              |          |
|    total_timesteps | 1000     |
---------------------------------
New best mean reward!
-----------------------------
| time/              |      |
|    episodes        | 12   |
|    fps             | 679  |
|    time_elapsed    | 1    |
|    total_timesteps | 1180 |
-----------------------------
-----------------------------
| time/              |      |
|    episodes        | 16   |
|    fps             | 679  |
|    time_elapsed    | 1    |
|    total_timesteps | 1180 |
-----------------------------
-----------------------------
| time/              |      |
|    episodes        | 20   |
|    fps             | 679  |
|    time_elapsed    | 1    |
|    total_timesteps | 1180 |
-----------------------------
------------------------